In [1]:
from lohrasb.best_estimator import BaseModel
import xgboost
from optuna.pruners import HyperbandPruner
from optuna.samplers._tpe.sampler import TPESampler
from sklearn.model_selection import KFold,train_test_split
import pandas as pd
import numpy as np
import optuna
from sklearn.pipeline import Pipeline
from feature_engine.imputation import (
    CategoricalImputer,
    MeanMedianImputer
    )
from category_encoders import OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score)
from sklearn.metrics import f1_score, mean_absolute_error
from sklearn.linear_model import *
from sklearn.svm import *
from xgboost import *
from sklearn.linear_model import *
from catboost import *
from lightgbm import *
from sklearn.neural_network import *
from imblearn.ensemble import *
from sklearn.ensemble import *



/Users/hjavedani/Documents/Lohrasb/.venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2022-09-30 14:43:24,292] A new study created in memory with name: no-name-63e6ebe0-cbfa-4bb0-aad3-799842a79767
[I 2022-09-30 14:43:24,294] A new study created in memory with name: no-name-97dacee6-b560-4d0d-a952-9e05603ea983


# Example 1 : Use Adult Data Set (a classification problem)
  
https://archive.ics.uci.edu/ml/datasets/Adult

In [2]:
urldata= "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
# column names
col_names=["age", "workclass", "fnlwgt" , "education" ,"education-num",
"marital-status","occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week",
"native-country","label"
]
# read data
data = pd.read_csv(urldata,header=None,names=col_names,sep=',')
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


# Define labels


In [3]:
data.loc[data['label']=='<=50K','label']=0
data.loc[data['label']==' <=50K','label']=0

data.loc[data['label']=='>50K','label']=1
data.loc[data['label']==' >50K','label']=1

data['label']=data['label'].astype(int)

# Train test split

In [4]:
X = data.loc[:, data.columns != "label"]
y = data.loc[:, data.columns == "label"]
y = y.values.ravel()


X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.33, random_state=42)


# Find feature types for later use

In [5]:
int_cols =  X_train.select_dtypes(include=['int']).columns.tolist()
float_cols =  X_train.select_dtypes(include=['float']).columns.tolist()
cat_cols =  X_train.select_dtypes(include=['object']).columns.tolist()


# Define model and set it argumens 

In [6]:
estimator = LGBMClassifier()
estimator_params = {
        "eval_metric": ["auc"],
        "max_depth": [4, 5],
        
    }


In [7]:
obj = BaseModel.bestmodel_factory.using_optuna(
            estimator=estimator,
            estimator_params=estimator_params,
            measure_of_accuracy="f1_score",
            add_extra_args_for_measure_of_accuracy = False,
            verbose=3,
            n_jobs=-1,
            random_state=42,
            # optuna params
            # optuna study init params
            study=optuna.create_study(
                storage=None,
                sampler=TPESampler(),
                pruner=HyperbandPruner(),
                study_name=None,
                direction="maximize",
                load_if_exists=False,
                directions=None,
            ),
            # optuna optimization params
            study_optimize_objective=None,
            study_optimize_objective_n_trials=10,
            study_optimize_objective_timeout=600,
            study_optimize_n_jobs=-1,
            study_optimize_catch=(),
            study_optimize_callbacks=None,
            study_optimize_gc_after_trial=False,
            study_optimize_show_progress_bar=False,
        )

[I 2022-09-30 14:43:25,293] A new study created in memory with name: no-name-8267daa6-e2f9-4fcf-82c9-5d798c6d18d3


# Build sklearn pipeline

In [8]:


pipeline =Pipeline([
            # int missing values imputers
            ('intimputer', MeanMedianImputer(
                imputation_method='median', variables=int_cols)),
            # category missing values imputers
            ('catimputer', CategoricalImputer(variables=cat_cols)),
            #
            ('catencoder', OrdinalEncoder()),
            # classification model
            ('obj', obj),


 ])
 



# Run Pipeline

In [9]:
pipeline.fit(X_train,y_train)
y_preds = pipeline.predict(X_test)
pred_labels = np.rint(y_preds)




Initializing Optuna
labels=None Not a float
labels=None Not a int
average='binary' Not a float
average='binary' Not a int
sample_weight=None Not a float
sample_weight=None Not a int
zero_division='warn' Not a float
zero_division='warn' Not a int
labels=None Not a float
labels=None Not a int
average='binary' Not a float
average='binary' Not a int
sample_weight=None Not a float
sample_weight=None Not a int
zero_division='warn' Not a float
zero_division='warn' Not a int
labels=None Not a float
labels=None Not a int
average='binary' Not a float
average='binary' Not a int
sample_weight=None Not a float
sample_weight=None Not a int
zero_division='warn' Not a float
zero_division='warn' Not a int
labels=None Not a float
labels=None Not a int
average='binary' Not a float
average='binary' Not a int
sample_weight=None Not a float
sample_weight=None Not a int
zero_division='warn' Not a float
zero_division='warn' Not a int
labels=None Not a float
labels=None Not a int
average='binary' Not a float
a

/Users/hjavedani/Documents/Lohrasb/.venv/lib/python3.10/site-packages/optuna/study/study.py:393: FutureWarning: `n_jobs` argument has been deprecated in v2.7.0. This feature will be removed in v4.0.0. See https://github.com/optuna/optuna/releases/tag/v2.7.0.
  warnings.warn(
[I 2022-09-30 14:43:27,235] Trial 6 finished with value: 0.6993630573248407 and parameters: {'eval_metric': 'auc', 'max_depth': 4}. Best is trial 6 with value: 0.6993630573248407.
[I 2022-09-30 14:43:27,246] Trial 7 finished with value: 0.6993630573248407 and parameters: {'eval_metric': 'auc', 'max_depth': 4}. Best is trial 6 with value: 0.6993630573248407.
[I 2022-09-30 14:43:27,253] Trial 3 finished with value: 0.6993630573248407 and parameters: {'eval_metric': 'auc', 'max_depth': 4}. Best is trial 6 with value: 0.6993630573248407.


{'eval_metric': 'auc', 'max_depth': 4}
[LightGBM] [Warning] Unknown parameter: eval_metric
{'eval_metric': 'auc', 'max_depth': 4}
[LightGBM] [Warning] Unknown parameter: eval_metric


[I 2022-09-30 14:43:27,472] Trial 0 finished with value: 0.6993630573248407 and parameters: {'eval_metric': 'auc', 'max_depth': 4}. Best is trial 6 with value: 0.6993630573248407.
[I 2022-09-30 14:43:27,478] Trial 1 finished with value: 0.6993630573248407 and parameters: {'eval_metric': 'auc', 'max_depth': 4}. Best is trial 6 with value: 0.6993630573248407.
[I 2022-09-30 14:43:27,799] Trial 5 finished with value: 0.7101449275362318 and parameters: {'eval_metric': 'auc', 'max_depth': 5}. Best is trial 5 with value: 0.7101449275362318.
[I 2022-09-30 14:43:27,809] Trial 4 finished with value: 0.7101449275362318 and parameters: {'eval_metric': 'auc', 'max_depth': 5}. Best is trial 5 with value: 0.7101449275362318.
[I 2022-09-30 14:43:27,819] Trial 2 finished with value: 0.7101449275362318 and parameters: {'eval_metric': 'auc', 'max_depth': 5}. Best is trial 5 with value: 0.7101449275362318.
[I 2022-09-30 14:43:27,987] Trial 9 finished with value: 0.6993630573248407 and parameters: {'eval_m

[LightGBM] [Warning] Unknown parameter: eval_metric
LGBMClassifier(eval_metric='auc', max_depth=5)
Initializing Optuna
labels=None Not a float
labels=None Not a int
average='binary' Not a float
average='binary' Not a int
sample_weight=None Not a float
sample_weight=None Not a int
zero_division='warn' Not a float
zero_division='warn' Not a int
labels=None Not a float
labels=None Not a int
average='binary' Not a float
average='binary' Not a int
sample_weight=None Not a float
sample_weight=None Not a int
zero_division='warn' Not a float
zero_division='warn' Not a int
labels=None Not a float
labels=None Not a int
average='binary' Not a float
average='binary' Not a int
sample_weight=None Not a float
sample_weight=None Not a int
zero_division='warn' Not a float
zero_division='warn' Not a int
labels=None Not a float
labels=None Not a int
average='binary' Not a float
average='binary' Not a int
sample_weight=None Not a float
sample_weight=None Not a int
zero_division='warn' Not a float
zero_div

[I 2022-09-30 14:43:31,257] Trial 17 finished with value: 0.7101449275362318 and parameters: {'eval_metric': 'auc', 'max_depth': 5}. Best is trial 5 with value: 0.7101449275362318.
[I 2022-09-30 14:43:31,279] Trial 11 finished with value: 0.7101449275362318 and parameters: {'eval_metric': 'auc', 'max_depth': 5}. Best is trial 5 with value: 0.7101449275362318.
[I 2022-09-30 14:43:31,294] Trial 14 finished with value: 0.7101449275362318 and parameters: {'eval_metric': 'auc', 'max_depth': 5}. Best is trial 5 with value: 0.7101449275362318.
[I 2022-09-30 14:43:31,305] Trial 12 finished with value: 0.7101449275362318 and parameters: {'eval_metric': 'auc', 'max_depth': 5}. Best is trial 5 with value: 0.7101449275362318.
[I 2022-09-30 14:43:31,318] Trial 16 finished with value: 0.7101449275362318 and parameters: {'eval_metric': 'auc', 'max_depth': 5}. Best is trial 5 with value: 0.7101449275362318.
[I 2022-09-30 14:43:31,320] Trial 10 finished with value: 0.7101449275362318 and parameters: {'

{'eval_metric': 'auc', 'max_depth': 5}
{'eval_metric': 'auc', 'max_depth': 5}
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: eval_metric


[I 2022-09-30 14:43:31,810] Trial 18 finished with value: 0.7101449275362318 and parameters: {'eval_metric': 'auc', 'max_depth': 5}. Best is trial 5 with value: 0.7101449275362318.
[I 2022-09-30 14:43:31,817] Trial 19 finished with value: 0.7101449275362318 and parameters: {'eval_metric': 'auc', 'max_depth': 5}. Best is trial 5 with value: 0.7101449275362318.


# Check performance of the pipeline

In [10]:
print('F1 score : ')
print(f1_score(y_test,pred_labels))
print('Classification report : ')
print(classification_report(y_test,pred_labels))
print('Confusion matrix : ')
print(confusion_matrix(y_test,pred_labels))


F1 score : 
0.7053015668598412
Classification report : 
              precision    recall  f1-score   support

           0       0.89      0.94      0.92      8196
           1       0.78      0.64      0.71      2550

    accuracy                           0.87     10746
   macro avg       0.84      0.79      0.81     10746
weighted avg       0.87      0.87      0.87     10746

Confusion matrix : 
[[7730  466]
 [ 907 1643]]
